In [27]:
import numpy as np
import pandas as pd

# from src.dataloader_ import *
# from src.network_ import *
# from src.utils import *

import os
import sys
import gc
import pickle
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import StratifiedKFold

train_series_dir = "../../../inputs/series_train.parquet/"
test_series_dir = "../../../inputs/series_test.parquet/"

data_dic_path = "../../../inputs/data_dictionary.csv"
sample_submission_path = "../../../inputs/sample_submission.csv"
train_path = "../../../inputs/train.csv"
test_path = "../../../inputs/test.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sample_submission = pd.read_csv(sample_submission_path)
data_dic = pd.read_csv(data_dic_path)

import os
import random

import numpy as np
import torch


def seed_torch(seed=1029):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


nb_name = os.path.basename(os.getcwd())  # notebook name
seed_torch(seed=42)

In [28]:
from glob import glob

# train_series_dir内のディレクトリ名を全て取得
has_timeseries_ids = []
paths = glob(f"{train_series_dir}/*")
for path in paths:
    dir_ = path.split("/")[-1]
    id_ = dir_.split("=")[-1]
    has_timeseries_ids.append(id_)

In [29]:
# siiラベル無し
non_sii = train[train["sii"].isnull()].reset_index(drop=True)
non_sii["has_time"] = 0

# siiラベル有り
has_sii = train[train["sii"].notnull()].reset_index(drop=True)
has_sii["has_time"] = has_sii["id"].apply(lambda x: 1 if x in has_timeseries_ids else 0)

In [30]:
has_sii["group_col"] = (
    has_sii["sii"].astype(str) + "_" + has_sii["has_time"].astype(str)
)
has_sii

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,has_time,group_col
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0,0,2.0_0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0,0,0.0_0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0,0,0.0_0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0,1,1.0_1
4,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,...,1.0,34.0,Summer,40.0,56.0,Spring,0.0,1.0,1,1.0_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,ff6c2bb8,Fall,8,0,NaN,NaN,Fall,17.139810,52.5,67.2,...,1.0,22.0,Fall,41.0,58.0,Fall,2.0,0.0,0,0.0_0
2732,ff759544,Summer,7,1,NaN,NaN,Summer,13.927006,48.5,46.6,...,0.0,33.0,Summer,48.0,67.0,Summer,0.0,1.0,0,1.0_0
2733,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,...,0.0,32.0,Winter,35.0,50.0,Fall,1.0,1.0,0,1.0_0
2734,ffcd4dbd,Fall,11,0,Spring,68.0,Winter,21.441500,60.0,109.8,...,1.0,31.0,Winter,56.0,77.0,Fall,0.0,1.0,1,1.0_1


In [45]:
import pickle

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_index, valid_index) in enumerate(
    skf.split(has_sii, has_sii["group_col"])
):
    t = has_sii.loc[train_index]
    v = has_sii.loc[valid_index]
    fold_train_ids = t["id"].values
    print(len(fold_train_ids), len(non_sii))
    fold_train_ids = list(fold_train_ids) + list(non_sii["id"].values)
    fold_valid_ids = v["id"].values

    with open(f"../divided-datasets/fold_train_ids_{fold}.pkl", "wb") as f:
        pickle.dump(fold_train_ids, f)

    with open(f"../divided-datasets/fold_valid_ids_{fold}.pkl", "wb") as f:
        pickle.dump(fold_valid_ids, f)

2188 1224
2189 1224
2189 1224
2189 1224
2189 1224
